In [16]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import pymysql
import sqlalchemy
from sqlalchemy import create_engine, text

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Gulim'
plt.rcParams['font.size'] = 12
plt.rcParams['figure.dpi'] = 100

In [17]:
# 광역시별 거리두기 단계
# https://namu.wiki/w/%EC%82%AC%ED%9A%8C%EC%A0%81%20%EA%B1%B0%EB%A6%AC%EB%91%90%EA%B8%B0/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD
cities = ['su','bs','dg','ic','gj','dj','us','jj','sj']
zip_code = ['11','21','22','23','24','25','26','39','29']

distance = pd.read_excel("사회적 거리두기 시행내역(임시).xlsx")
distance = distance.iloc[:21]
distance = distance.T.reset_index()

for i,j in zip(cities,zip_code):
    globals()[i+'_distance'] = distance.iloc[[1],:].T.drop('index')
    
    globals()[i+'_distance_20'] = globals()[i+'_distance'].iloc[:11]
    globals()[i+'_distance_21'] = globals()[i+'_distance'].iloc[11:]
    globals()[i+'_distance_20'].insert(0,'local_id',j)
    globals()[i+'_distance_20'].insert(0,'Month',['02','03','04','05','06','07','08','09','10','11','12'])
    globals()[i+'_distance_20'].insert(0,'Year','2020')
    globals()[i+'_distance_21'].insert(0,'local_id',j)
    globals()[i+'_distance_21'].insert(0,'Month',['01','02','03','04','05','06','07','08','09','10'])
    globals()[i+'_distance_21'].insert(0,'Year','2021')
    globals()[i+'_distance'] = pd.concat([globals()[i+'_distance_20'],globals()[i+'_distance_21']])

distance_final = pd.concat([su_distance,bs_distance,dg_distance,ic_distance,gj_distance,dj_distance,us_distance,sj_distance,jj_distance])
distance_final.rename(columns = {1:'단계'},inplace=True)


In [18]:
distance_final.to_csv('distance.csv',index=False)

In [19]:
# 년도별 시간대별 평균 전력거래량
#https://www.data.go.kr/data/15104294/fileData.do
by_time = pd.read_csv("한국전력거래소_시간대별 평균 전력거래량_20211231.csv",encoding = 'euc-kr')
by_time.columns = (['Year','Time','평균전력거래량'])

In [20]:
by_time.to_csv('by_time.csv',index=False)

In [21]:
# 년도별 월별 분류별 총합
#https://kosis.kr/statHtml/statHtml.do?orgId=310&tblId=DT_31002N_A004&vw_cd=MT_ZTITLE&list_id=U_15&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE
by_month_sum = pd.read_csv('월별(2017~2022).csv',encoding='euc-kr')
by_month_sum = by_month_sum.T
by_month_sum = by_month_sum.drop(['2017','2017.4','2017.5','2017.6','2018','2018.4','2018.5','2018.6','2019','2019.4','2019.5','2019.6','2020','2020.4','2020.5','2020.6','2021','2021.4','2021.5','2021.6','2022','2022.4','2022.5','2022.6'])

month_hm = by_month_sum.loc[by_month_sum[0] == '가정용',:]
month_hm.drop([1],axis=1,inplace=True)
month_hm.columns=(['분류','합계','1','2','3','4','5','6','7','8','9','10','11','12'])
month_hm = month_hm.iloc[:,2:]
month_hm.reset_index(inplace=True)

month_ofc = by_month_sum.loc[(by_month_sum[0] == '공공용') | (by_month_sum[0] == '서비스업 및 기타')]
month_ofc.drop([1],axis=1,inplace=True)
month_ofc.index=(['2017','2017','2018','2018','2019','2019','2020','2020','2021','2021','2022','2022'])
tmp1 = month_ofc.iloc[:,0]
tmp2 = month_ofc.iloc[:,1:].astype(int)
month_ofc = pd.concat([tmp1,tmp2],axis=1)
month_ofc = month_ofc.groupby(month_ofc.index).sum().iloc[:,2:]
month_ofc.columns=(['1','2','3','4','5','6','7','8','9','10','11','12'])
month_ofc.reset_index(inplace=True)

year = ['2017','2018','2019','2020','2021','2022']
for i,j in zip(year,range(month_hm.shape[0])):
    globals()['month_hm'+i] = month_hm.T.iloc[:,[j]].drop('index').reset_index()
    globals()['month_ofc'+i] = month_ofc.T.iloc[:,[j]].drop('index')
    globals()['month_hm'+i].insert(0,'Year',i)
    globals()['month_hm'+i].columns = (['Year','Month','가정용']) 
    globals()['month_ofc'+i].columns = (['업무용'])

by_month_sum_final = pd.concat([month_hm2017,month_hm2018,month_hm2019,month_hm2020,month_hm2021,month_hm2022])
month_ofc_sum = pd.concat([month_ofc2017,month_ofc2018,month_ofc2019,month_ofc2020,month_ofc2021,month_ofc2022])
by_month_sum_final.index=(np.arange(by_month_sum_final.shape[0]))
month_ofc_sum.index=(np.arange(month_ofc_sum.shape[0]))
by_month_sum_final = pd.concat([by_month_sum_final,month_ofc_sum],axis=1)
by_month_sum_final.rename(columns = {'가정용':'주거용'},inplace=True)

In [22]:
by_month_sum_final.to_csv('by_month_sum.csv',index=False)

In [23]:
# 년도별 월별 분류별 판매전력량
# https://home.kepco.co.kr/kepco/KO/ntcob/list.do?boardCd=BRD_000097&menuCd=FN05030101
year2 = ['19','20','21','22']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in year2:
    for j in month:
        globals()['sale_'+i+j] = pd.read_excel(i+j+'.xlsx')
    for k in month[:6]:
        globals()['sale_23'+k] = pd.read_excel('23'+k+'.xlsx')

In [24]:
# 19년도 ~ 20년도
for i in year2[:2]:
    for j in month:
        globals()['sale_'+i+j] = globals()['sale_'+i+j].loc[(globals()['sale_'+i+j].index == 0) | (globals()['sale_'+i+j].index == 7) | (globals()['sale_'+i+j].index == 40)]
        globals()['sale_'+i+j] = globals()['sale_'+i+j].iloc[:,3:20]
        globals()['sale_'+i+j].index = (['가정용','업무용','산업용'])
        globals()['sale_'+i+j] = globals()['sale_'+i+j].astype(int)
        globals()['sale_'+i+j] = globals()['sale_'+i+j].T
# 21년도 01월 ~ 03월
for k in month[:3]:
    globals()['sale_21'+k] = globals()['sale_21'+k].loc[(globals()['sale_21'+k].index == 0) | (globals()['sale_21'+k].index == 7) | (globals()['sale_21'+k].index == 40)]
    globals()['sale_21'+k] = globals()['sale_21'+k].iloc[:,3:20]
    globals()['sale_21'+k].index = (['가정용','업무용','산업용'])
    globals()['sale_21'+k] = globals()['sale_21'+k].astype(int)
    globals()['sale_21'+k] = globals()['sale_21'+k].T
# 21년도 04월 ~ 05월 (4월부터 형식 변화)
for l in month[3:]:
    tmp1 = pd.DataFrame(globals()['sale_21'+l].iloc[0].drop([globals()['sale_21'+l].columns[0],globals()['sale_21'+l].columns[1]])).T
    tmp2 = pd.DataFrame(globals()['sale_21'+l].iloc[1:3].drop([globals()['sale_21'+l].columns[0],globals()['sale_21'+l].columns[1]],axis=1).sum()).T
    tmp3 = pd.DataFrame(globals()['sale_21'+l].iloc[3:6].drop([globals()['sale_21'+l].columns[0],globals()['sale_21'+l].columns[1]],axis=1).sum()).T
    globals()['sale_21'+l] = pd.concat([tmp1,tmp2,tmp3])
    globals()['sale_21'+l] = globals()['sale_21'+l].iloc[:,:17]
    globals()['sale_21'+l].index = (['가정용','업무용','산업용'])
    globals()['sale_21'+l] = globals()['sale_21'+l].astype(int)
    globals()['sale_21'+l] = globals()['sale_21'+l].T
# 22년도 (22년도부터 컬럼 순서 변화)
for m in month:
    tmp1 = pd.DataFrame(globals()['sale_22'+m].iloc[0].drop([globals()['sale_22'+m].columns[0],globals()['sale_22'+m].columns[1]])).T
    tmp2 = pd.DataFrame(globals()['sale_22'+m].iloc[1:3].drop([globals()['sale_22'+m].columns[0],globals()['sale_22'+m].columns[1]],axis=1).sum()).T
    tmp3 = pd.DataFrame(globals()['sale_22'+m].iloc[3:6].drop([globals()['sale_22'+m].columns[0],globals()['sale_22'+m].columns[1]],axis=1).sum()).T
    globals()['sale_22'+m] = pd.concat([tmp1,tmp2,tmp3])
    globals()['sale_22'+m] = globals()['sale_22'+m].iloc[:,:17]
    globals()['sale_22'+m].index = (['가정용','업무용','산업용'])
    globals()['sale_22'+m] = globals()['sale_22'+m].astype(int)
    globals()['sale_22'+m] = globals()['sale_22'+m].T
# 23년도 (상반기 데이터만 존재)
for n in month[:6]:
    tmp1 = pd.DataFrame(globals()['sale_23'+n].iloc[0].drop([globals()['sale_23'+n].columns[0],globals()['sale_23'+n].columns[1]])).T
    tmp2 = pd.DataFrame(globals()['sale_23'+n].iloc[1:3].drop([globals()['sale_23'+n].columns[0],globals()['sale_23'+n].columns[1]],axis=1).sum()).T
    tmp3 = pd.DataFrame(globals()['sale_23'+n].iloc[3:6].drop([globals()['sale_23'+n].columns[0],globals()['sale_23'+n].columns[1]],axis=1).sum()).T
    globals()['sale_23'+n] = pd.concat([tmp1,tmp2,tmp3])
    globals()['sale_23'+n] = globals()['sale_23'+n].iloc[:,:17]
    globals()['sale_23'+n].index = (['가정용','업무용','산업용'])
    globals()['sale_23'+n] = globals()['sale_23'+n].astype(int)
    globals()['sale_23'+n] = globals()['sale_23'+n].T
# 컬럼 이름 정리 및 각 도시 zip code로 바꾸기
for o in year2[:3]:
    for p in month:
        globals()['sale_'+o+p].index=([11,21,22,23,24,25,26,31,32,33,34,35,36,37,38,39,29])
        globals()['sale_'+o+p].reset_index(inplace=True)
        globals()['sale_'+o+p].rename(columns = {'index':'local_id'},inplace=True)
        globals()['sale_'+o+p].insert(0,'Month',p)
        globals()['sale_'+o+p].insert(0,'Year','20'+o)
for q in month:
    globals()['sale_22'+q].index=([11,21,22,23,24,25,26,29,31,32,33,34,35,36,37,38,39])
    globals()['sale_22'+q].reset_index(inplace=True)
    globals()['sale_22'+q].rename(columns = {'index':'local_id'},inplace=True)
    globals()['sale_22'+q].insert(0,'Month',q)
    globals()['sale_22'+q].insert(0,'Year','2022')
for r in month[:6]:
    globals()['sale_23'+r].index=([11,21,22,23,24,25,26,29,31,32,33,34,35,36,37,38,39])
    globals()['sale_23'+r].reset_index(inplace=True)
    globals()['sale_23'+r].rename(columns = {'index':'local_id'},inplace=True)
    globals()['sale_23'+r].insert(0,'Month',r)
    globals()['sale_23'+r].insert(0,'Year','2023')

sale_by_month_final = pd.concat([sale_1901,sale_1902,sale_1903,sale_1904,sale_1905,sale_1906,sale_1907,sale_1908,sale_1909,sale_1910,sale_1911,sale_1912,sale_2001,sale_2002,sale_2003,sale_2004,sale_2005,sale_2006,sale_2007,sale_2008,sale_2009,sale_2010,sale_2011,sale_2012,sale_2101,sale_2102,sale_2103,sale_2104,sale_2105,sale_2106,sale_2107,sale_2108,sale_2109,sale_2110,sale_2111,sale_2112,sale_2201,sale_2202,sale_2203,sale_2204,sale_2205,sale_2206,sale_2207,sale_2208,sale_2209,sale_2210,sale_2211,sale_2212,sale_2301,sale_2302,sale_2303,sale_2304,sale_2305,sale_2306])
sale_by_month_final.rename(columns = {'가정용':'주거용'},inplace=True)

In [25]:
sale_by_month_final.to_csv('month.csv',index=False)

In [26]:
# 데이터 이관
# DB 접속
protocol ="mysql+pymysql://"
user = "root"
password = "reaver813"
host = "localhost"
port = "3306"
db = "mini1"
con_str = f"{protocol}{user}:{password}@{host}:{port}/{db}" 
conn = create_engine(con_str)

In [27]:
# 테이블 생성
ct = """
CREATE TABLE IF NOT EXISTS `by_time` (
`num` int NOT NULL AUTO_INCREMENT,
`연도` text NOT NULL,
`시간` text NOT NULL,
`평균전력거래량` int NOT NULL,
PRIMARY KEY (`num`));
"""
ct2 = """
CREATE TABLE IF NOT EXISTS `by_month_sum` (
`num` int NOT NULL AUTO_INCREMENT,
`연도` text NOT NULL,
`월` text NOT NULL,
`주거용` int NOT NULL,
`업무용` int NOT NULL,
PRIMARY KEY (`num`));
"""
ct3 = """
CREATE TABLE IF NOT EXISTS `month` (
`num` int NOT NULL AUTO_INCREMENT,
`연도` text NOT NULL,
`월` text NOT NULL,
`local_id` INT NOT NULL, 
`주거용` int NOT NULL,
`업무용` int NOT NULL,
`산업용` int NOT NULL,
PRIMARY KEY (`num`),
KEY `FK_month_loc` (`local_id`),
CONSTRAINT `FK_month_loc` FOREIGN KEY (`local_id`) REFERENCES `loc` (`local_id`));
"""
ct4 = """
CREATE TABLE IF NOT EXISTS `distance` (
`num` int NOT NULL AUTO_INCREMENT,
`연도` text NOT NULL,
`월` text NOT NULL,
`local_id` INT NOT NULL, 
`단계` float NOT NULL,
PRIMARY KEY (`num`),
KEY `FK_distance_loc` (`local_id`),
CONSTRAINT `FK_distance_loc` FOREIGN KEY (`local_id`) REFERENCES `loc` (`local_id`)); 
"""
conn.execute(ct)
conn.execute(ct2)
conn.execute(ct3)
conn.execute(ct4)

In [28]:
# 테이블에 데이터 추가
iv = """
INSERT INTO distance(
연도,
월,
local_id,
단계)
VALUES(
:Year,
:Month,
:local_id,
:단계)
"""
for i in distance_final.to_dict('records'):
    conn.execute(text(iv),i)
iv2 = """
INSERT INTO by_month_sum(
연도,
월,
주거용,
업무용)
VALUES(
:Year,
:Month,
:주거용,
:업무용)
"""
for i in by_month_sum_final.to_dict('records'):
    conn.execute(text(iv2),i)
iv3 = """
INSERT INTO by_time(
연도,
시간,
평균전력거래량)
VALUES(
:Year,
:Time,
:평균전력거래량)
"""
for i in by_time.to_dict('records'):
    conn.execute(text(iv3),i)
iv4 = """
INSERT INTO month(
연도,
월,
local_id,
주거용,
업무용,
산업용)
VALUES(
:Year,
:Month,
:local_id,
:주거용,
:업무용,
:산업용)
"""
for i in sale_by_month_final.to_dict('records'):
    conn.execute(text(iv4),i)